In [6]:
import os
import subprocess
import zipfile
import cairosvg
import pandas as pd
import json
import cv2
import re
import shutil
import json

In [7]:
def setup_directories():
    # setup directories
    try:
        os.mkdir('res')
        print('created /res directory')
    except OSError:
        print("res/ already exists")
setup_directories()

res/ already exists


In [8]:
autodraw_url = 'https://storage.googleapis.com/autodraw-assets/Autodraw_illustrations.zip'
# fetch Autodraw
autodraw_name = 'Autodraw_illustrations.zip'

if len([file for file in os.listdir('res/') if autodraw_name in file]) == 0:
    subprocess.call(
        ["wget",
         autodraw_url,
        "--directory-prefix=res"])
else:
    print("autodraw is already downloaded: {}".
            format([file for file in os.listdir('res/') if autodraw_name in file]))

autodraw is already downloaded: ['Autodraw_illustrations.zip']


In [9]:
# unzip
fontawesome_dir = ['res/' + file for file in os.listdir('res/') if
                    autodraw_name in file and '.zip' in file][0]

with zipfile.ZipFile(fontawesome_dir, "r") as zip_ref:
    zip_ref.extractall("res")

autodraw_dir = ['res/' + file for file in os.listdir('res/') if
                    'Autodraw' in file and 'zip' not in file][0]

print('Autodraw files saved in: {}'.format(autodraw_dir))

Autodraw files saved in: res/Autodraw_illustrations


In [10]:
# the only rich library is 'Selman'
if 'Selman' not in autodraw_dir:
    autodraw_dir = os.path.join(autodraw_dir, 'Selman')
    print('Autodraw directory changed to: {}'.format(autodraw_dir))
else:
    print('Autodraw directory is: {}'.format(autodraw_dir))

Autodraw directory changed to: res/Autodraw_illustrations/Selman


In [11]:
# read svg file -> png data
WIDTH = 512
HEIGHT = 512
png_dir = 'res/autodraw-png'

try:
    os.mkdir(png_dir)
except OSError:
    print("{} already exists".format(png_dir))

files = sorted(os.listdir(autodraw_dir))

filenames = [file.split('.svg')[0] for file in files]
textdescrip = [file.replace('-', ' ') for file in filenames]
filenames_png = [file + '.png' for file in filenames]

res/autodraw-png already exists


In [12]:
for file, filename in zip(files, filenames):
    # convert svg to png and resize
    cairosvg.svg2png(url=os.path.join(autodraw_dir, file),
                     output_width=WIDTH, 
                     output_height=HEIGHT,
                    write_to=os.path.join(png_dir, filename + '.png'))
    # convert to 3 channels and black and white
    img = cv2.imread(os.path.join(png_dir, filename + '.png'),
                        cv2.IMREAD_UNCHANGED)

    if len(img.shape) and img.shape[2] == 4:

        # change black -> white and white -> black
        img[:, :, 0] = 255-img[:, :, 3]
        img[:, :, 1] = 255-img[:, :, 3]
        img[:, :, 2] = 255-img[:, :, 3]
        # remove 4th color channel (4th is alpha channel)
        img = img[:, :, :3]

        cv2.imwrite(os.path.join(png_dir, filename + '.png'), img)
    else:
        print("Image does not have 4 channels; deleting it: {}".
                format(os.path.join(png_dir, filename + '.png')))
        os.unlink(os.path.join(png_dir, filename + '.png'))

In [13]:
# re-arrange into the right directory structure
# res/autodraw-png/ -> res/autodraw-png-instance/<instance_name>/<filename>.png

# create instance directory
instance_dir = 'res/autodraw-png-instance'
try:
    os.mkdir(instance_dir)
except OSError:
    print("{} already exists".format(instance_dir))

files = sorted(os.listdir(png_dir))
filenames = [file.split('.png')[0] for file in files]

# find unique instances in filenames
# split on any non-alphanumeric character and keep everything before the last split
instances = [re.split('[^a-zA-Z0-9]', filename)[:-1] for filename in filenames]
instances = [' '.join(instance) for instance in instances]
# remove dashes from instances
instances = [instance.replace('-', ' ') for instance in instances]
# make directories for each instance
for instance in list(set(instances)):
    try:
        os.mkdir(os.path.join(instance_dir, instance))
    except OSError:
        print("{} already exists".format(os.path.join(instance_dir, instance)))
# copy instances over to the right directory
for file, instance in zip(files, instances):
    #instance = re.split('[^a-zA-Z0-9]', file.split('.png')[0])[0]
    shutil.copy(os.path.join(png_dir, file), os.path.join(instance_dir, instance))

res/autodraw-png-instance already exists
res/autodraw-png-instance/pond already exists
res/autodraw-png-instance/toucan already exists
res/autodraw-png-instance/backpack already exists
res/autodraw-png-instance/tennis player already exists
res/autodraw-png-instance/hippo already exists
res/autodraw-png-instance/paper clip already exists
res/autodraw-png-instance/knight shield already exists
res/autodraw-png-instance/pool already exists
res/autodraw-png-instance/owl already exists
res/autodraw-png-instance/popsicle already exists
res/autodraw-png-instance/shrimp already exists
res/autodraw-png-instance/boomerang already exists
res/autodraw-png-instance/candle already exists
res/autodraw-png-instance/thumbs up already exists
res/autodraw-png-instance/snail already exists
res/autodraw-png-instance/football player already exists
res/autodraw-png-instance/the eiffel tower already exists
res/autodraw-png-instance/bell already exists
res/autodraw-png-instance/flamingo already exists
res/autod

In [14]:
# create a concepts_list.json file
# this file will follow the format from here: https://github.com/ShivamShrirao/diffusers/blob/main/examples/dreambooth/concepts_list.json

# create a list of all the instances
instances = sorted(os.listdir(instance_dir))

concept_lists = []
for instance in instances:
    # instance_token = re.split('[^a-zA-Z0-9]', instance)[0].replace('-', ' ')
    # general format for each instance
    concept_list = {
        "instance_prompt": "black and white {} icon".format(instance),
        "instance_data_dir": os.path.join(instance_dir, instance),
        "class_prompt": "black and white icon",
        "class_data_dir": png_dir, # this is the same for all instances; all autodraw icons
    }
    concept_lists.append(concept_list)

In [16]:
# write to json file with pretty formatting
with open('res/concepts_list.json', 'w') as f:
    json.dump(concept_lists, f, indent=4)